- Đơn giản hóa bài toán: tìm cách tối ưu cho 31 ngày; hết 1 vòng thì lặp lại theo đúng thứ tự cũ

- Cần tìm **thuật toán** để: thay đổi cách sắp xếp 1 cách hợp lý theo từng bước -> tăng giá trị fitness function

In [1]:
import numpy as np
import random

In [2]:
# A = [1] * 3 + [2] * 6 + [3] * 6 + [4] * 4
# B = [1] * 2 + [2] * 4 + [3] * 4 + [4] * 7
# C = [1] * 7 + [2] * 7 + [3] * 8 + [4] * 0
# D = [1] * 2 + [2] * 4 + [3] * 5 + [4] * 11
A = [1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
B = [1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4]
C = [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
D = [1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
A_s = (A * 2)[:31]
B_s = (B * 2)[:31]
C_s = (C * 2)[:31]
D_s = (D * 2)[:31]

select_ex = {
    'BS1': A_s,
    'BS2': B_s,
    'DD1': C_s,
    'DD2': D_s
}

In [3]:
import pandas as pd

In [4]:
dfex = pd.DataFrame({
    'BS1': A_s,
    'BS2': B_s,
    'DD1': C_s,
    'DD2': D_s
})
dfex

,BS1,BS2,DD1,DD2
0,1,1,1,1
1,1,1,1,1
2,1,2,1,2
3,2,2,1,2
4,2,2,1,2
5,2,2,1,2
6,2,3,1,3
7,2,3,2,3
8,2,3,2,3
9,3,3,2,3


In [5]:
A_ori = np.array([1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4])
B_ori = np.array([1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4])
C_ori = np.array([1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3])
D_ori = np.array([1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
popu_size = 50 #number of selections (each citizen is a selection)
cross_change = 5 #change {n} bad points 
mutate_num = 3
days_num = 31   #solve selection of 31 days

num_iter = 100
expect_score = 90

eliminate_rate = 0.3           
 ## how the population will grow after selection. For now it will be growing to the original popu num (0.3 is in respect to orgininal number)
crossover_rate = eliminate_rate   

columns = ['BS1', 'BS2', 'DD1', 'DD2']

In [146]:
# does np.random.shuffle give completely random shuffling? Or are there any elements repetitive?

def fitness(selection):
    A_s = selection['BS1']
    B_s = selection['BS2']
    C_s = selection['DD1']
    D_s = selection['DD2']
    score = 0
    lA = len(A_s)
    lB = len(B_s)
    lC = len(C_s)
    lD = len(D_s)
    lmin = min(lA, lB, lC, lD)
    for i in range(days_num):
        if i >= lmin:
            score += row_score(A_s[i % lA], B_s[i % lB], C_s[i % lC], D_s[i % lD])
        else: 
            score += row_score(A_s[i], B_s[i], C_s[i], D_s[i])
    return score

def row_score(a, b, c, d):
    thisset = {a, b, c, d}
    return len(thisset)

# find a row with score less than 4:
## best_row_score default to be 4 in this problem
## temp_score as return value --- only for testing now. DELETE soon
def row_low_score(selection, best_row_score=4):
    A_s = selection['BS1']
    B_s = selection['BS2']
    C_s = selection['DD1']
    D_s = selection['DD2']
    lA = len(A_s)
    lB = len(B_s)
    lC = len(C_s)
    lD = len(D_s)
    
    iteration = 0 ##to ensure that this does not run forever
    
    while (iteration <= days_num * 2):
        index = random.randint(0, days_num)
        temp_score = row_score(A_s[index % lA], B_s[index % lB], C_s[index % lC], D_s[index % lD])
        if temp_score < best_row_score:
            return index   #temp_score only for testing now
        iteration = iteration + 1
    
    #in case not found any row with low score
    return 0

def rand_column_name():
    rand_key = random.randint(0, len(columns) - 1)
    return columns[rand_key]

def rand_column_index(column):
    rand_id = random.randint(0, len(column) - 1)
    return rand_id

# a column, i.e. BS1 or BS2 or DD1 or DD2
def gen_column(a1=1, a2=1, a3=1, a4=1):
    # A = [1] * 3 + [2] * 6 + [3] * 6 + [4] * 4
    col = np.array([1] * a1 + [2] * a2 + [3] * a3 + [4] * a4)
    np.random.shuffle(col)
    return col

def gen_selection():
    select = {}
    #numbers are hard-coded here (because of this specific problem). Change here if needed.
    select['BS1'] = gen_column(3, 6, 6, 4)
    select['BS2'] = gen_column(2, 4, 4, 7)
    select['DD1'] = gen_column(7, 7, 8, 0)
    select['DD2'] = gen_column(2, 4, 5, 11)
    return select

# for each selection, keep info of just A, B, C, D (raw - NOT A_s, B_s, C_s, D_s -> )
def gen_population(popu_size=10):
    popu = []
    
    for i in range(popu_size):
        popu = popu + [gen_selection()]
    
    return popu

def population_fitness(popu):
    return list(map(lambda sel: fitness(sel), popu))

def mean_fitness(popu):
    return sum(population_fitness(popu)) / len(popu)

def best_fitness(popu):
    return max(population_fitness(popu))

# eliminate 30% ({eliminate_rate}) worst-performing selections
def selection(popu, eliminate_rate=0.1):
    indices_eli = list(worst_indices(population_fitness(popu), eliminate_rate))
    indices_eli.sort(reverse=True)
    
    for i in range(len(indices_eli)):
        popu.pop(indices_eli[i])

# worst selections for eliminating(by index)
def worst_indices(lst_fitness, eliminate_rate=0.1):
    lst_sorted = np.argsort(lst_fitness)
    eliminate_num = int(len(lst_fitness) * eliminate_rate)
    return lst_sorted[:eliminate_num]

#######################
######DONE crossover!

#how to crossover? last elements are repeated elements -> very tricky
#how to ensure crossover will increase (NOT decrease)
# CROSSOVER method: change {n} bad points of a selection
# but that way: selections not learning from each other (but maybe no problem. )
# crossover_rate: ratio that the popuation row back. For now use `origin_population` for simplicity
def crossover(popu, origin_population=50, bound_iter=4000):
    iteration = 0   # to ensure that don't run forever in case cannot find anything else
    while (len(popu) < origin_population and iteration <= bound_iter):   #should have used crossover_rate, but use popu_size for now (for simplicity)
        rand_sel_id = random.randint(0, len(popu) - 1)
        new_sel = indi_breeding(popu[rand_sel_id])
        if not (new_sel is None):
            popu.append(new_sel)
            iteration = 0
        else:
            iteration = iteration + 1
            

#indi_breeding: choose 'not good' row i -> randomly pick a column j -> swap_cell for (i, j) and (i', j), with i' picked randomly (if this i' not work -> pick another i')
## if swap_cell makes score increase -> approve; if not disapprove
## indi_breeding run repeatedly until approving
#breeding_strength is how much a selection changes during breeding #for now just use default = 1
def indi_breeding(sel, breeding_strength=1):
    bad_row = row_low_score(sel)    #ignoring case `bad_row = 0` for ease of coding
    col_rand = rand_column_name()     
    bad_row = bad_row % len(sel[col_rand]) # bad_row can be greater than 22, so need modulo-divide first
    iteration = 0  #iteration makes thing haunt when there is no expected answer (probabilistically)
    while (iteration <= len(sel[col_rand])):   
        id_rand = rand_column_index(sel[col_rand])
        if sel[col_rand][id_rand] != sel[col_rand][bad_row]:
            new_col = swap_cell(sel[col_rand], bad_row, id_rand)
            new_sel = sel.copy()
            new_sel[col_rand] = new_col
            #implement condition: STRICTLY GREATER than old selection
            if fitness(new_sel) > fitness(sel):
                return new_sel    
        iteration = iteration + 1
    #in case can not change anything to increase score
    return None


# a cell is a position in a column
def swap_cell(column_origin, index_from=0, index_to=0):
    column = column_origin.copy()
    temp = column[index_to]
    column[index_to] = column[index_from]
    column[index_from] = temp
    return column

def swap_cell_origin(column_origin, index_from=0, index_to=0):
    temp = column_origin[index_to]
    column_origin[index_to] = column_origin[index_from]
    column_origin[index_from] = temp
    return column_origin

#################################
####### CONTINUE: mutating 

# mutate_rate means how many selections get mutated
def mutate(popu, mutate_rate=0.1, mutate_strength=0.1):
    num_mutate = int(len(popu) * mutate_rate)
    count = 0
    
    while (count < num_mutate):
        rand_sel_id = random.randint(0, len(popu) - 1)
        indi_mutate(popu[rand_sel_id], mutate_strength)    #second param for indi_mutate is hardcoded - change here if needed
        count += 1
        
# mutate_strength means how many pairs get swaped in a selection
def indi_mutate(sel, mutate_strength=0.1):
    num_swap = int( sum(list(map(lambda k: len(sel[k]), sel))) * mutate_strength )
    count = 0
    
    while (count < num_swap):
        col_rand = rand_column_name()     
        id1_rand = rand_column_index(sel[col_rand])
        id2_rand = id1_rand
        while (sel[col_rand][id1_rand] == sel[col_rand][id2_rand]):
            id2_rand = rand_column_index(sel[col_rand])
        
        swap_cell_origin(sel[col_rand], id1_rand, id2_rand)
        count += 1        

########################################
# done selection and crossover. Testing with them
# run genetic algorithm
def run_GA(popu_size=50, eliminate_rate=0.3, bound_iter=4000, mutate_rate=0.3, mutate_strength=0.1, aim=100):
    popu = gen_population(popu_size)
    prev_score = 0
    score = 1
    count = 0 #for testing
    while (best_fitness(popu) < aim):
        selection(popu, eliminate_rate)
#         prev_score = mean_fitness(popu)
        crossover(popu, popu_size, bound_iter)
#         score = mean_fitness(popu)
        mutate(popu, mutate_rate, mutate_strength)   # 2nd param is hardcoded - temporarily for case popu_size=10
        count += 1
    return (popu,count)

In [148]:
ans, count = run_GA(popu_size=50, eliminate_rate=0.3, bound_iter=4000, mutate_rate=0, mutate_strength=0, aim=115)
population_fitness(ans)

[113,
 113,
 113,
 113,
 113,
 113,
 114,
 113,
 114,
 114,
 113,
 113,
 114,
 114,
 113,
 114,
 113,
 114,
 114,
 114,
 113,
 114,
 113,
 114,
 113,
 113,
 114,
 114,
 113,
 113,
 114,
 113,
 114,
 113,
 114,
 114,
 114,
 114,
 114,
 114,
 115,
 114,
 114,
 114,
 114,
 114,
 114,
 114,
 114,
 114]

In [149]:
count

29

In [95]:
eliminate_rate

0.3

In [ ]:
sum(population_fitness(ans)) / len(ans)

In [ ]:
population_fitness(ans)

In [34]:
mean_fitness(ans)

89.7

In [18]:
indi_mutate(popu_ex[0], 0.026)
popu_ex[0]

{'BS1': array([3, 2, 2, 4, 3, 3, 2, 3, 1, 4, 1, 4, 2, 2, 2, 3, 3, 1, 4]),
 'BS2': array([4, 4, 4, 2, 1, 4, 4, 4, 3, 3, 4, 2, 2, 2, 3, 3, 1]),
 'DD1': array([3, 2, 3, 3, 1, 2, 2, 3, 2, 2, 1, 3, 1, 2, 1, 3, 1, 3, 1, 3, 1, 2]),
 'DD2': array([4, 3, 4, 4, 4, 2, 3, 1, 4, 2, 3, 3, 4, 4, 2, 4, 3, 1, 4, 4, 4, 2])}

In [64]:
popu_ex = gen_population(10)
popu_ex

[{'BS1': array([3, 2, 1, 3, 2, 4, 4, 2, 4, 3, 2, 4, 1, 3, 1, 2, 3, 3, 2]),
  'BS2': array([3, 4, 4, 4, 4, 3, 4, 1, 2, 2, 3, 1, 4, 2, 4, 3, 2]),
  'DD1': array([3, 3, 3, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 3, 3, 2, 3, 1, 1, 1, 3, 2]),
  'DD2': array([4, 4, 1, 4, 3, 2, 4, 2, 4, 3, 4, 2, 3, 3, 4, 4, 3, 2, 4, 4, 4, 1])},
 {'BS1': array([2, 2, 3, 3, 1, 3, 3, 4, 2, 2, 1, 2, 4, 3, 3, 2, 1, 4, 4]),
  'BS2': array([3, 2, 3, 2, 3, 4, 4, 2, 2, 4, 4, 1, 4, 1, 4, 3, 4]),
  'DD1': array([1, 3, 3, 2, 3, 3, 1, 1, 3, 3, 2, 1, 1, 2, 3, 2, 1, 2, 3, 2, 2, 1]),
  'DD2': array([4, 1, 4, 4, 2, 3, 4, 3, 2, 4, 2, 3, 3, 4, 2, 4, 3, 4, 1, 4, 4, 4])},
 {'BS1': array([2, 2, 4, 2, 3, 1, 1, 2, 3, 2, 3, 3, 4, 4, 3, 1, 2, 3, 4]),
  'BS2': array([4, 3, 3, 4, 4, 3, 2, 4, 3, 2, 2, 1, 4, 1, 4, 4, 2]),
  'DD1': array([1, 2, 1, 1, 3, 2, 1, 2, 1, 1, 1, 3, 2, 3, 2, 2, 3, 3, 3, 2, 3, 3]),
  'DD2': array([4, 4, 3, 4, 3, 2, 3, 4, 4, 3, 4, 2, 4, 4, 1, 4, 4, 2, 3, 2, 1, 4])},
 {'BS1': array([1, 3, 2, 3, 2, 4, 3, 4, 1, 2, 2, 2, 2, 4, 3, 

In [65]:
population_fitness(popu_ex)

[85, 93, 89, 85, 86, 85, 84, 83, 84, 88]

In [29]:
best_fitness(popu_ex)

91

In [66]:
sum(population_fitness(popu_ex)) / 10

86.2

In [91]:
selection(popu_ex, eliminate_rate)
# sum(population_fitness(popu_ex)) / 7
mean_fitness(popu_ex)

100.42857142857143

In [92]:
# def crossover(popu, crossover_rate=0.1):
crossover(popu_ex, 10)
# sum(population_fitness(popu_ex)) / 10
mean_fitness(popu_ex)

100.8

In [ ]:
population_fitness(popu_ex)

In [21]:
popu_ex_small = gen_population(2)
popu_ex_small

[{'BS1': array([3, 2, 1, 4, 3, 3, 3, 4, 3, 1, 3, 4, 2, 2, 4, 1, 2, 2, 2]),
  'BS2': array([3, 4, 4, 2, 1, 4, 2, 2, 2, 3, 3, 3, 4, 4, 1, 4, 4]),
  'DD1': array([1, 3, 2, 3, 2, 2, 1, 2, 1, 1, 1, 3, 2, 3, 1, 3, 3, 3, 1, 3, 2, 2]),
  'DD2': array([2, 4, 4, 3, 3, 2, 4, 4, 4, 4, 1, 4, 4, 3, 3, 4, 4, 3, 1, 2, 4, 2])},
 {'BS1': array([3, 2, 3, 3, 2, 1, 2, 1, 4, 2, 2, 4, 2, 4, 3, 3, 3, 1, 4]),
  'BS2': array([3, 3, 1, 1, 4, 2, 3, 4, 4, 4, 3, 4, 2, 2, 2, 4, 4]),
  'DD1': array([1, 2, 3, 1, 2, 1, 3, 2, 2, 3, 2, 1, 2, 3, 3, 3, 2, 1, 1, 3, 1, 3]),
  'DD2': array([3, 2, 3, 4, 3, 4, 4, 4, 1, 2, 4, 4, 3, 2, 3, 4, 4, 4, 4, 4, 2, 1])}]

In [23]:
mutate(popu_ex_small, 0.5)

In [24]:
popu_ex_small

[{'BS1': array([3, 2, 1, 4, 3, 3, 3, 4, 2, 1, 3, 4, 2, 2, 4, 1, 2, 2, 3]),
  'BS2': array([3, 4, 4, 2, 1, 4, 2, 2, 2, 3, 3, 3, 4, 4, 1, 4, 4]),
  'DD1': array([1, 3, 2, 3, 2, 2, 1, 2, 1, 1, 1, 3, 2, 3, 1, 3, 3, 3, 1, 3, 2, 2]),
  'DD2': array([2, 4, 4, 3, 3, 4, 4, 4, 4, 4, 1, 2, 4, 3, 3, 4, 4, 3, 1, 2, 4, 2])},
 {'BS1': array([3, 2, 3, 3, 2, 1, 2, 1, 4, 2, 2, 4, 2, 4, 3, 3, 3, 1, 4]),
  'BS2': array([3, 3, 1, 1, 4, 2, 3, 4, 4, 4, 3, 4, 2, 2, 2, 4, 4]),
  'DD1': array([1, 2, 3, 1, 2, 1, 3, 2, 2, 3, 2, 1, 2, 3, 3, 3, 2, 1, 1, 3, 1, 3]),
  'DD2': array([3, 2, 3, 4, 3, 4, 4, 4, 1, 2, 4, 4, 3, 2, 3, 4, 4, 4, 4, 4, 2, 1])}]

In [25]:
mutate(popu_ex_small, 1)
popu_ex_small

[{'BS1': array([3, 2, 1, 4, 3, 3, 3, 4, 2, 1, 3, 4, 2, 2, 4, 1, 2, 2, 3]),
  'BS2': array([3, 4, 3, 2, 1, 4, 2, 2, 2, 3, 3, 4, 4, 4, 1, 4, 4]),
  'DD1': array([1, 3, 2, 2, 3, 2, 1, 2, 1, 1, 1, 3, 2, 3, 1, 3, 3, 3, 1, 3, 2, 2]),
  'DD2': array([2, 4, 4, 3, 3, 4, 4, 4, 4, 4, 1, 2, 4, 3, 3, 4, 4, 3, 1, 2, 4, 2])},
 {'BS1': array([3, 2, 2, 3, 2, 1, 2, 1, 4, 2, 3, 4, 2, 4, 3, 3, 3, 1, 4]),
  'BS2': array([3, 3, 1, 1, 4, 2, 3, 4, 4, 4, 3, 4, 2, 2, 2, 4, 4]),
  'DD1': array([1, 2, 3, 1, 2, 1, 3, 2, 2, 3, 2, 1, 2, 3, 3, 3, 2, 1, 1, 3, 1, 3]),
  'DD2': array([3, 3, 3, 4, 2, 4, 4, 4, 1, 2, 4, 4, 3, 2, 3, 4, 4, 4, 4, 4, 2, 1])}]